In [2]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import numpy as np
import matplotlib.pyplot as plt
import os, shutil
import urllib.request
# from app import app
from flask import Flask, request, redirect, jsonify
from werkzeug.utils import secure_filename


app = Flask(__name__)
ALLOWED_EXTENSIONS = set(['png', 'jpg'])

# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 2

#defining classes

classes=['non-suspicious','suspicious']

app.config['UPLOAD_FOLDER'] = '/home/raghava/Downloads/sample/unknown'

data_dir = '/home/raghava/Downloads/sample'

extension = ['.jpg', '.png']

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/file-upload', methods=['POST'])
def upload_file():
    # check if the post request has the file part
    if 'file' not in request.files:
        resp = jsonify({'message' : 'No file part in the request'})
        resp.status_code = 400
        return resp
    file = request.files['file']
    if file.filename == '':
        resp = jsonify({'message' : 'No file selected for uploading'})
        resp.status_code = 400
        return resp
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        if filename[-4:] == '.png':
            ext = '.png'
        else:
            ext = '.jpg'
        for File in os.listdir(app.config['UPLOAD_FOLDER']):
            if File.endswith('.jpg') or File.endswith('.png'):
                # print(File)
                os.remove(os.path.join(app.config['UPLOAD_FOLDER'], File))
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        src = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        dest = os.path.join(app.config['UPLOAD_FOLDER'], filename[:-4] + '_1' + ext)
        shutil.copy(src, dest)
        
        test_transforms = transforms.Compose([transforms.Resize(255),

                                      transforms.CenterCrop(224),

                                      transforms.ToTensor(),

                                      transforms.Normalize([0.485, 0.456, 0.406],

                                                           [0.229, 0.224, 0.225])])


        test_data = datasets.ImageFolder(data_dir, transform=test_transforms)

        test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,
                                                  num_workers=num_workers)

        model = models.densenet121(pretrained=True)

        # Freeze parameters so we don't backprop through them
        for param in model.parameters():
            param.requires_grad = False
        # print('1')    
        model.classifier = nn.Sequential(nn.Linear(1024, 256),
                                         nn.ReLU(),
                                         nn.Dropout(0.2),
                                         nn.Linear(256, 2),
                                         nn.LogSoftmax(dim=1))

        # Only train the classifier parameters, feature parameters are frozen
        optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)


        model.load_state_dict(torch.load('model_first.pt',map_location=torch.device('cpu')))
        model.eval()

        test_loss = 0.0
        class_correct = list(0. for i in range(10))
        class_total = list(0. for i in range(10))

        for data, target in test_loader:
            output = model(data)
            # print(output.tolist())
            na = output.to('cpu').detach().numpy()
            print((na))
            resp = jsonify({'message' : 'File successfully uploaded','output':output.tolist()})
        resp.status_code = 201
        return resp
    else:
        resp = jsonify({'message' : 'Allowed file types are png, jpg'})
        resp.status_code = 400
        return resp

if __name__ == "__main__":
    app.run()
    

# API to hit at postman for

#http://127.0.0.1:5000/file-upload

# in postman body select file in key section and upload a image in the value section

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
